In [ ]:
# 1. csv파일을 load

# 2. csv파일의 '영엄장명' column을 읽어서 네이버 지도검색 url에서 넣어서 검색

# 3. 메뉴테그를 순서대로 읽은 뒤 대표메뉴 3가지와 가격을 추출 후 df에 저장
# column: 'menu_1_name, menu_1_price, menu_2_name, menu_2_price, menu_3_name, menu_3_price'

# 4 df를 csv파일로 저장

In [ ]:
import os
from time import sleep
import tqdm

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# CSV 파일 로드
df = pd.read_csv('../featureSelector/data/c4_general_cafeteria_chuncheon_CC_opened_SS-HJ_sampling.csv', encoding='utf-8')
print("CSV load complete")

# 결과를 저장할 DataFrame 초기화 column: 'index_cafeteria, cafeteria_name, menu_name, menu_price'
result_df = pd.DataFrame({
    'index_cafeteria': [],
    'cafeteria_name': [],
    'menu_name': [],
    'menu_price': []
})

state_num = 0

In [ ]:
test_range = range(len(df))
# test_range = [12, 17, 21, 32, 34, 55, 92] # 테스트용 변수 설정
for i in tqdm.tqdm(test_range):
    print("index 변수 설정 완료 i = ", i)
    KEYWORD_INDEX = df['번호'][i]
    KEYWORD = df['사업장명'][i]
    URL = "https://m.map.kakao.com/actions/searchView?q=" + "춘천 " + KEYWORD

    driver = webdriver.Chrome()  # 웹 드라이버 생성
    driver.get(URL)  # 링크 열기
    sleep(1)
    serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
    serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱

    try:
        ########-- todo: 영업시간과 연락처가 있는 경우 수집 --########
        ########--          insert code !          --########
        
        sleep(0.7)
        button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
        button.click()
        
    except NoSuchElementException: # 검색어가 없는 경우 
        try:
            print("해당 요소를 찾을 수 없습니다: .tit_g")
            driver.quit()  # 웹 드라이버 종료
            
            URL = "https://m.map.kakao.com/actions/searchView?q=" + KEYWORD

            driver = webdriver.Chrome()  # 웹 드라이버 생성
            driver.get(URL)  # 링크 열기
            sleep(1.5)
            serch_html = driver.page_source  # 검색 페이지의 HTML 가져오기
            serch_html_soup = BeautifulSoup(serch_html, 'lxml')  # BeautifulSoup으로 파싱
            
            
            ########-- todo: 영업시간과 연락처가 있는 경우 수집 --########
            ########--          insert code !          --########
            
            sleep(0.7)
            button = driver.find_element(By.CLASS_NAME, 'tit_g')  # 클래스 이름 'tit_g'를 가진 버튼 찾고 클릭
            button.click()
        except NoSuchElementException: # 그래도 검색어가 없는 경우
            driver.quit()  # 웹 드라이버 종료
            state_num = 1

    # 검색어가 없을 경우(state_num==1일떄) 아래의 작업을 하지 않음.
    if state_num == 1:
        driver.quit()
        continue
    
    sleep(0.7)
    menu_html = driver.page_source  # 페이지의 HTML 가져오기
    menu_html_soup = BeautifulSoup(menu_html, 'lxml')  # BeautifulSoup으로 파싱
    sleep(0.7)

    # DataFrame 초기화
    extracted_df = pd.DataFrame({
        'index_cafeteria': [KEYWORD_INDEX] * 3,
        'cafeteria_name': [KEYWORD] * 3
    })

    # 'name_menu' 클래스를 가진 모든 요소 찾기
    name_menus = menu_html_soup.find_all(class_='name_menu', limit=3)
    data_name = [menu.get_text() for menu in name_menus]

    # 'price_menu' 클래스를 가진 모든 요소 찾기
    price_menus = menu_html_soup.find_all(class_='price_menu', limit=3)
    data_price = [menu.get_text().replace('가격: ', '').replace('\n', '') for menu in price_menus]

    # 데이터 길이 확인 및 DataFrame 업데이트
    if len(data_name) == len(extracted_df):
        extracted_df['menu_name'] = data_name
        print("상위 3개 메뉴 추출 완료")
    else:
        print("메뉴 데이터의 길이가 맞지 않아 추가하지 않습니다.")

    if len(data_price) == len(extracted_df):
        extracted_df['menu_price'] = data_price
        print("메뉴별 가격 추출 완료")
    else:
        print("가격 데이터의 길이가 맞지 않아 추가하지 않습니다.")
        
    driver.quit()  # 웹 드라이버 종료

    result_df = pd.concat([result_df, extracted_df]) # result_df에 extracted_df의 1~3행을 추가

In [ ]:
# result_df의 중복된 행을 제거
result_df = result_df.drop_duplicates()

# result_df의 index를 초기화
result_df = result_df.reset_index(drop=True)

In [ ]:
# result_df의 NAN값의 비율을 column별로 출력
print(result_df.isnull().sum() / len(result_df))

In [ ]:
# result_df의 값을 저장
result_df.to_csv('../featureSelector/data/c4_general_cafeteria_chuncheon_CC_opened_SS-HJ_sampling_menu-price.csv', encoding='utf-8', index=False)